In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [23]:
books=pd.read_csv('BX-Books.csv',sep=';',error_bad_lines=False,encoding='latin-1')
user=pd.read_csv('BX-Users.csv',sep=';',error_bad_lines=False,encoding='latin-1')
rating=pd.read_csv('BXBookRatings.csv',sep=';',error_bad_lines=False,encoding='latin-1')
book_rating=pd.merge(rating,books, on='ISBN')
cols=['Year-Of-Publication','Publisher','Book-Author','Image-URL-S', 'Image-URL-M', 'Image-URL-L']
book_rating.drop(cols,axis=1,inplace=True)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [24]:
book_rating.head(5)

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0,Flesh Tones: A Novel
1,2313,034545104X,5,Flesh Tones: A Novel
2,6543,034545104X,0,Flesh Tones: A Novel
3,8680,034545104X,5,Flesh Tones: A Novel
4,10314,034545104X,9,Flesh Tones: A Novel


In [35]:
rating_count=(book_rating.
              groupby(by=['Book-Title']) ['Book-Rating'].
                                  count().reset_index().
                                  rename(columns={'Book-Rating':'Ratingcount_book'})[['Book-Title','Ratingcount_book']])
rating_count.head()

,Book-Title,Ratingcount_book
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [36]:
threshold=25
rating_count=rating_count.query('Ratingcount_book>=@threshold')
rating_count.head()

,Book-Title,Ratingcount_book
75,'Salem's Lot,47
203,10 Lb. Penalty,61
422,101 Dalmatians,37
673,"14,000 Things to Be Happy About",28
697,16 Lighthouse Road,65


In [37]:
user_rating=pd.merge(rating_count,book_rating,left_on='Book-Title',right_on='Book-Title',how='left')

In [38]:
user_rating.head()

,Book-Title,Ratingcount_book,User-ID,ISBN,Book-Rating
0,'Salem's Lot,47,8936,067103975X,0
1,'Salem's Lot,47,172245,067103975X,0
2,'Salem's Lot,47,189835,067103975X,5
3,'Salem's Lot,47,9226,0451168089,0
4,'Salem's Lot,47,33283,0451168089,10


In [41]:
user_count=(user_rating.groupby(by=['User-ID'])['Book-Rating'].count().
            reset_index().
            rename(columns={'Book-Rating':'RatingCount_User'})
           [['User-ID','RatingCount_User']])
user_count.head()

,User-ID,RatingCount_User
0,8,2
1,9,2
2,10,1
3,14,1
4,16,2


In [43]:
threshold=20
user_count=user_count.query('RatingCount_User>=@threshold')
user_count.head()

,User-ID,RatingCount_User
52,243,68
54,254,139
89,487,21
96,507,61
115,638,51


In [44]:
combined=user_rating.merge(user_count,left_on='User-ID',right_on='User-ID',how='inner')

In [49]:
combined.head()


,Book-Title,Ratingcount_book,User-ID,ISBN,Book-Rating,RatingCount_User
0,'Salem's Lot,47,8936,067103975X,0,177
1,1st to Die: A Novel,509,8936,0446610038,0,177
2,A Case of Need,236,8936,0451210638,0,177
3,A Perfect Stranger,54,8936,0440168724,0,177
4,Accident,126,8936,0440217547,0,177


In [48]:
combined.shape

(263467, 6)

In [50]:
print('Number of Unique Books',combined['Book-Title'].nunique())
print('Number of Unique Users',combined['User-ID'].nunique())

Number of Unique Books 5850
Number of Unique Users 3192


In [56]:
scalar=MinMaxScaler()
combined['Book-Rating']=combined['Book-Rating'].values.astype(float)
rating_scaled=pd.DataFrame(scalar.fit_transform(combined['Book-Rating'].values.reshape(-1,1)))
combined['Book-Rating']=rating_scaled


MinMaxScaler(copy=True, feature_range=(0, 1))

In [57]:
combined = combined.drop_duplicates(['User-ID', 'Book-Title'])
user_book_matrix = combined.pivot(index='User-ID', columns='Book-Title', values='Book-Rating')
user_book_matrix.fillna(0, inplace=True)

users = user_book_matrix.index.tolist()
books = user_book_matrix.columns.tolist()

user_book_matrix = user_book_matrix.as_matrix()

C:\Users\partheban\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [58]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [59]:
num_input = combined['Book-Title'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

In [60]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

In [61]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X

In [62]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [63]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

In [65]:
with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(user_book_matrix.shape[0] / batch_size)
    user_book_matrix = np.array_split(user_book_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_book_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    user_book_matrix = np.concatenate(user_book_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_book_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='Book-Rating')
    pred_data.columns = ['User-ID', 'Book-Title', 'Book-Rating']
    pred_data['User-ID'] = pred_data['User-ID'].map(lambda value: users[value])
    pred_data['Book-Title'] = pred_data['Book-Title'].map(lambda value: books[value])
    
    keys = ['User-ID', 'Book-Title']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['User-ID', 'Book-Rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('User-ID').head(10)

epoch: 1 Loss: 0.34574684271445644
epoch: 2 Loss: 0.2912343539051957
epoch: 3 Loss: 0.06859219526350088
epoch: 4 Loss: 0.0036622070564100375
epoch: 5 Loss: 0.003264013157164739
epoch: 6 Loss: 0.0031615673157560464
epoch: 7 Loss: 0.002907276121582222
epoch: 8 Loss: 0.002752337898997651
epoch: 9 Loss: 0.0027359464835211796
epoch: 10 Loss: 0.0027249954745767043
epoch: 11 Loss: 0.0027171584168592324
epoch: 12 Loss: 0.0027112945898917023
epoch: 13 Loss: 0.0027067564468789885
epoch: 14 Loss: 0.0027031493245286272
epoch: 15 Loss: 0.0027002191073824088
epoch: 16 Loss: 0.0026977951817501048
epoch: 17 Loss: 0.002695758710615337
epoch: 18 Loss: 0.0026940246122878985
epoch: 19 Loss: 0.00269253081437405
epoch: 20 Loss: 0.00269123088762901
epoch: 21 Loss: 0.0026900896439226445
epoch: 22 Loss: 0.0026890798062495476
epoch: 23 Loss: 0.0026881800117585196
epoch: 24 Loss: 0.002687373193451664
epoch: 25 Loss: 0.002686645721781303
epoch: 26 Loss: 0.0026859864630930865
epoch: 27 Loss: 0.0026853862717481597


In [68]:
top_ten_ranked.loc[top_ten_ranked['User-ID'] == 8936]

,User-ID,Book-Title,Book-Rating
496155,8936,The Lovely Bones: A Novel,0.075052
495702,8936,The Da Vinci Code,0.062238
496460,8936,The Secret Life of Bees,0.047146
493237,8936,Harry Potter and the Chamber of Secrets (Book 2),0.046067
493241,8936,Harry Potter and the Prisoner of Azkaban (Book 3),0.043939
492102,8936,Bridget Jones's Diary,0.042179
493806,8936,Life of Pi,0.041953
493243,8936,Harry Potter and the Sorcerer's Stone (Harry P...,0.040076
496801,8936,To Kill a Mockingbird,0.038220
496384,8936,The Red Tent (Bestselling Backlist),0.038177


In [69]:
book_rating.loc[book_rating['User-ID'] == 8936].sort_values(by=['Book-Rating'], ascending=False)

,User-ID,ISBN,Book-Rating,Book-Title
411297,8936,0590479075,10,The Case of the Secret Message (Clue Jr.)
399900,8936,1551669455,10,Deadly Grace
411301,8936,0590479083,10,The Case of the Stolen Jewel (Clue Jr.)
411794,8936,1551660482,10,Guilt By Silence
270517,8936,0399142282,10,Hornet's Nest
...,...,...,...,...
263794,8936,0807506451,0,"Benny Uncovers a Mystery (The Boxcar Children,..."
263786,8936,0807503673,0,The Animal Shelter Mystery (Boxcar Children My...
263781,8936,0807503193,0,The Amusement Park Mystery (Boxcar Children My...
255047,8936,0312263120,0,Digital Fortress : A Thriller


In [25]:
#books.columns
rating.columns
#user.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')